# 랭체인 멀티턴

- LangChain의 RunnableWithMessageHistory를 사용하여 대화기록을 관리하는 챗봇 구현
- 세션 ID별로 대화 기록이 어떻게 분리되어 저장되는지 확인

## 라이브러리 불러오기

In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

c:\Users\user\miniconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## API KEY 불러오기

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

## 대화 기록 함수 정의

In [4]:
model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

## 첫 번째 대화

In [5]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 김정은이야.")],
    config=config,
)

print(response.content)

안녕하세요, 김정은님! 어떻게 도와드릴까요?


## 두 번째 대화

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 김정은입니다. 다른 질문이나 도움이 필요하신 것이 있으신가요?


## 세션 테스트

In [7]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만 당신의 이름을 알 수 없습니다. 다른 질문이 있으면 도와드릴 수 있습니다!'

In [8]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름이 김정은이라고 말씀하신 것에 대해 이야기했습니다. 그 외에 다른 주제나 질문이 있으시면 말씀해 주세요!'

## 스트리밍 응답 출력

In [9]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="")
    # print(r.content, end="|")

김정은이라는 이름을 기반으로 볼 때, 당신은 북한의 사람일 가능성이 높습니다. 북한의 문화는 독특하고, 통제된 사회주의 체제를 바탕으로 형성되어 있습니다. 

북한 문화의 특징 중 일부는 다음과 같습니다:

1. **예술과 음악**: 북한은 선전의 일환으로 음악과 미술이 중요한 역할을 합니다. 체계적인 검열 아래에서 국가에 충성하는 작품들이 많이 만들어집니다.

2. **전통 의상**: 북한의 전통 의상인 "한복"은 특별한 날에 주로 입습니다.

3. **명절과 축제**: 김일성 일가를 기리는 다양한 기념일과 명절이 있으며, 이들 행사에는 대규모의 퍼포먼스와 전시가 포함되곤 합니다.

4. **가족 중심의 문화**: 가족 간의 유대가 매우 중요하며, 전통적으로 부모에 대한 존경이 강조됩니다.

5. **음식**: 북한 음식은 김치, 냉면, 그리고 다양한 형태의 찌개가 대표적입니다. 북한의 음식 문화는 일반적으로 소박하지만 건강한 식사를 중요시합니다.

더 궁금한 점이나 다른 나라에 대해 알고 싶다면 말씀해 주세요!